In [2]:
import pandas as pd
import requests
import json
import numpy as np
import geopandas as gpd

In [3]:
variable_code = '72295'

In [4]:
page_0 = f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_code}?unit-level=5&year=2000&year=2001&year=2002&year=2003&year=2004&year=2005&year=2006&year=2007&year=2008&year=2009&year=2010&year=2011&year=2012&year=2013&year=2014&year=2015&year=2016&year=2017&year=2018&year=2019&year=2020&year=2021&page-size=100&page=0'
page_1 = f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_code}?unit-level=5&year=2000&year=2001&year=2002&year=2003&year=2004&year=2005&year=2006&year=2007&year=2008&year=2009&year=2010&year=2011&year=2012&year=2013&year=2014&year=2015&year=2016&year=2017&year=2018&year=2019&year=2020&year=2021&page-size=100&page=1'
page_2 = f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_code}?unit-level=5&year=2000&year=2001&year=2002&year=2003&year=2004&year=2005&year=2006&year=2007&year=2008&year=2009&year=2010&year=2011&year=2012&year=2013&year=2014&year=2015&year=2016&year=2017&year=2018&year=2019&year=2020&year=2021&page-size=100&page=2'
page_3 = f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_code}?unit-level=5&year=2000&year=2001&year=2002&year=2003&year=2004&year=2005&year=2006&year=2007&year=2008&year=2009&year=2010&year=2011&year=2012&year=2013&year=2014&year=2015&year=2016&year=2017&year=2018&year=2019&year=2020&year=2021&page-size=100&page=3'

In [5]:
def get_df (page_url):
    response_API = requests.get(page_url)
    population_summary = json.loads(response_API.text)
    rows = []

    for data in population_summary['results']:
        for value in data['values']:
            row = {'id': data['id'],
                'name': data['name'],
                'year': value['year'],
                'value': value['val'],
                'powiat_numer': data['id'][2:4] + data['id'][7:9]}
            rows.append(row)
    df = pd.DataFrame(rows)
    return df

In [6]:
def get_population_df():
    df_0 = get_df(page_0)
    df_1 = get_df(page_1)
    df_2 = get_df(page_2)
    df_3 = get_df(page_3)
    dfs = [df_0, df_1, df_2, df_3]
    population_df = pd.concat(dfs, axis=0)

    return population_df    

In [7]:
population_df = get_population_df()

In [8]:
population_df

,id,name,year,value,powiat_numer
0,011212001000,Powiat bocheński,2000,49494,1201
1,011212001000,Powiat bocheński,2001,49594,1201
2,011212001000,Powiat bocheński,2002,49832,1201
3,011212001000,Powiat bocheński,2003,50118,1201
4,011212001000,Powiat bocheński,2004,50970,1201
...,...,...,...,...,...
1775,071427338000,Powiat żyrardowski,2017,39689,1438
1776,071427338000,Powiat żyrardowski,2018,39587,1438
1777,071427338000,Powiat żyrardowski,2019,39518,1438
1778,071427338000,Powiat żyrardowski,2020,39519,1438


In [9]:
def transform_population_df(population_df):
    population_df_copy = population_df.copy()
    population_df_copy['name_normalized'] = population_df['name'].str.replace('Powiat', '')
    population_df_copy['name_normalized'] = population_df_copy['name_normalized'].str.replace(' m.', '')
    population_df_copy['name_normalized'] = population_df_copy['name_normalized'].str.lstrip()
    population_df_copy['name_normalized'] = population_df_copy['name_normalized'].str.replace('Wałbrzych od 2013', 'Wałbrzych')
    population_df_copy['name_normalized'] = population_df_copy['name_normalized'].str.replace('Wałbrzych do 2002', 'Wałbrzych')
    population_df_copy['name_normalized'] = population_df_copy['name_normalized'].str.replace('st. Warszawa', 'Warszawa')
    population_df_copy['powiat_numer'] = population_df_copy['powiat_numer'].astype('int').astype('str')
    population_df_copy = population_df_copy.rename(columns={'value':'population'})
    return population_df_copy

In [10]:
population_df = transform_population_df(population_df)

In [15]:
population_df.query('name_normalized== "Wałbrzych"')

,id,name,year,population,powiat_numer,name_normalized
1054,030210363000,Powiat m. Wałbrzych do 2002,2000,69458,263,Wałbrzych
1055,030210363000,Powiat m. Wałbrzych do 2002,2001,68961,263,Wałbrzych
1056,030210363000,Powiat m. Wałbrzych do 2002,2002,68437,263,Wałbrzych
1057,030210365000,Powiat m. Wałbrzych od 2013,2013,62284,265,Wałbrzych
1058,030210365000,Powiat m. Wałbrzych od 2013,2014,61657,265,Wałbrzych
1059,030210365000,Powiat m. Wałbrzych od 2013,2015,61078,265,Wałbrzych
1060,030210365000,Powiat m. Wałbrzych od 2013,2016,60609,265,Wałbrzych
1061,030210365000,Powiat m. Wałbrzych od 2013,2017,60097,265,Wałbrzych
1062,030210365000,Powiat m. Wałbrzych od 2013,2018,59628,265,Wałbrzych
1063,030210365000,Powiat m. Wałbrzych od 2013,2019,58897,265,Wałbrzych


TODO na pozniej: sprawdzic Wałbrzych i wałbrzyski bo cos jest z nimi nie tak

In [20]:
deaths_path = "/Users/nelatoma/Documents/icm/magisterka/zgony/MAGISTERKA_OFFICIAL/results/all_poviats_stats_females.csv"
age_group = "TOTAL"

In [39]:
def get_total_df(deaths_path, age_group):
    ALL_DATA = pd.read_csv(deaths_path)
    poviats_data_Y = ALL_DATA.query('powiat_numer > 32')
    total_df = poviats_data_Y.query(f'wiek_eurostat == "{age_group}"')
    total_df_copy = total_df.copy()
    total_df_copy['name_normalized'] = total_df_copy['powiat_nazwa']
    total_df_copy = total_df_copy.rename(columns={'YEAR':'year', 'sum':'deaths'})
    total_df_copy['year'] = total_df_copy['year'].astype('str')
    total_df_copy['powiat_numer'] = total_df_copy['powiat_numer'].astype('str') 

    return total_df_copy

In [40]:
total_df = get_total_df(deaths_path=deaths_path, age_group=age_group)

In [41]:
total_df

,wiek,powiat_numer,powiat_nazwa,year,wiek_eurostat,deaths,name_normalized
2,Ogółem,201,bolesławiecki,2000,TOTAL,321,bolesławiecki
3,Ogółem,202,dzierżoniowski,2000,TOTAL,540,dzierżoniowski
4,Ogółem,203,głogowski,2000,TOTAL,280,głogowski
5,Ogółem,204,górowski,2000,TOTAL,151,górowski
6,Ogółem,205,jaworski,2000,TOTAL,206,jaworski
...,...,...,...,...,...,...,...
166672,Ogółem,3217,wałecki,2021,TOTAL,341,wałecki
166673,Ogółem,3218,łobeski,2021,TOTAL,253,łobeski
166674,Ogółem,3261,Koszalin,2021,TOTAL,745,Koszalin
166675,Ogółem,3262,Szczecin,2021,TOTAL,2972,Szczecin


In [42]:
total_df.query("powiat_nazwa=='Warszawa' or powiat_nazwa == 'warszawski'")
    

,wiek,powiat_numer,powiat_nazwa,year,wiek_eurostat,deaths,name_normalized
173,Ogółem,1431,warszawski,2000,TOTAL,8789,warszawski
7973,Ogółem,1431,warszawski,2001,TOTAL,8767,warszawski
15775,Ogółem,1431,warszawski,2002,TOTAL,8844,warszawski
15787,Ogółem,1465,Warszawa,2002,TOTAL,24,Warszawa
23745,Ogółem,1465,Warszawa,2003,TOTAL,8996,Warszawa
31665,Ogółem,1465,Warszawa,2004,TOTAL,8971,Warszawa
39585,Ogółem,1465,Warszawa,2005,TOTAL,8904,Warszawa
47505,Ogółem,1465,Warszawa,2006,TOTAL,8783,Warszawa
55425,Ogółem,1465,Warszawa,2007,TOTAL,8640,Warszawa
63345,Ogółem,1465,Warszawa,2008,TOTAL,8987,Warszawa


In [44]:
total_df.loc[15787, 'deaths'] = total_df.loc[15787]['deaths'] + total_df.loc[15775]['deaths']

In [47]:
total_df = total_df.query("powiat_nazwa != 'warszawski' or year != '2002'")

In [48]:
total_df.query("powiat_nazwa=='Warszawa' or powiat_nazwa == 'warszawski'")

,wiek,powiat_numer,powiat_nazwa,year,wiek_eurostat,deaths,name_normalized
173,Ogółem,1431,warszawski,2000,TOTAL,8789,warszawski
7973,Ogółem,1431,warszawski,2001,TOTAL,8767,warszawski
15787,Ogółem,1465,Warszawa,2002,TOTAL,8868,Warszawa
23745,Ogółem,1465,Warszawa,2003,TOTAL,8996,Warszawa
31665,Ogółem,1465,Warszawa,2004,TOTAL,8971,Warszawa
39585,Ogółem,1465,Warszawa,2005,TOTAL,8904,Warszawa
47505,Ogółem,1465,Warszawa,2006,TOTAL,8783,Warszawa
55425,Ogółem,1465,Warszawa,2007,TOTAL,8640,Warszawa
63345,Ogółem,1465,Warszawa,2008,TOTAL,8987,Warszawa
71265,Ogółem,1465,Warszawa,2009,TOTAL,9396,Warszawa


In [49]:

population_df.groupby("name_normalized").count().query("year < 22")

,id,name,year,population,powiat_numer
name_normalized,,,,,
Warszawa,20,20,20,20,20
Wałbrzych,12,12,12,12,12
brzeziński,20,20,20,20,20
gołdapski,20,20,20,20,20
leski,20,20,20,20,20
sztumski,20,20,20,20,20
warszawski,2,2,2,2,2
wschowski,20,20,20,20,20
węgorzewski,20,20,20,20,20


In [36]:
total_df.groupby("name_normalized").count().query("year < 22")

,wiek,powiat_numer,powiat_nazwa,year,wiek_eurostat,deaths
name_normalized,,,,,,
Warszawa,20,20,20,20,20,20
Wałbrzych,12,12,12,12,12,12
bieruńsko-lędziński,20,20,20,20,20,20
brzeziński,20,20,20,20,20,20
gołdapski,20,20,20,20,20,20
jeleniogórski,21,21,21,21,21,21
karkonoski,1,1,1,1,1,1
leski,20,20,20,20,20,20
olecki,20,20,20,20,20,20


In [50]:
population_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8336 entries, 0 to 1779
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               8336 non-null   object
 1   name             8336 non-null   object
 2   year             8336 non-null   object
 3   population       8336 non-null   int64 
 4   powiat_numer     8336 non-null   object
 5   name_normalized  8336 non-null   object
dtypes: int64(1), object(5)
memory usage: 455.9+ KB


In [51]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8336 entries, 2 to 166676
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   wiek             8336 non-null   object
 1   powiat_numer     8336 non-null   object
 2   powiat_nazwa     8336 non-null   object
 3   year             8336 non-null   object
 4   wiek_eurostat    8336 non-null   object
 5   deaths           8336 non-null   int64 
 6   name_normalized  8336 non-null   object
dtypes: int64(1), object(6)
memory usage: 521.0+ KB


In [56]:
def get_CMR_table(population_df, total_df):
    merged_df = pd.merge(population_df, total_df, on=['year', 'powiat_numer'], how='inner')
    merged_df = merged_df.rename(columns={'name_normalized_x':'name_normalized_pop', 'name_normalized_y':'name_normalized_tot'})
    merged_df['CMR'] = merged_df['deaths']/merged_df['population']*100000

    return merged_df

In [57]:
merged_df = get_CMR_table(population_df, total_df)

In [58]:
merged_df

,id,name,year,population,powiat_numer,name_normalized_pop,wiek,powiat_nazwa,wiek_eurostat,deaths,name_normalized_tot,CMR
0,011212001000,Powiat bocheński,2000,49494,1201,bocheński,Ogółem,bocheński,TOTAL,425,bocheński,858.689942
1,011212001000,Powiat bocheński,2001,49594,1201,bocheński,Ogółem,bocheński,TOTAL,417,bocheński,840.827519
2,011212001000,Powiat bocheński,2002,49832,1201,bocheński,Ogółem,bocheński,TOTAL,427,bocheński,856.879114
3,011212001000,Powiat bocheński,2003,50118,1201,bocheński,Ogółem,bocheński,TOTAL,386,bocheński,770.182370
4,011212001000,Powiat bocheński,2004,50970,1201,bocheński,Ogółem,bocheński,TOTAL,409,bocheński,802.432804
...,...,...,...,...,...,...,...,...,...,...,...,...
8331,071427338000,Powiat żyrardowski,2017,39689,1438,żyrardowski,Ogółem,żyrardowski,TOTAL,460,żyrardowski,1159.011313
8332,071427338000,Powiat żyrardowski,2018,39587,1438,żyrardowski,Ogółem,żyrardowski,TOTAL,457,żyrardowski,1154.419380
8333,071427338000,Powiat żyrardowski,2019,39518,1438,żyrardowski,Ogółem,żyrardowski,TOTAL,463,żyrardowski,1171.617997
8334,071427338000,Powiat żyrardowski,2020,39519,1438,żyrardowski,Ogółem,żyrardowski,TOTAL,542,żyrardowski,1371.492194


In [66]:
merged_df.to_csv('../results/CMR_total_females.csv', index=False)

In [67]:
merged_df.loc[merged_df['CMR'].idxmax()]


id                         062013805000
name                   Powiat hajnowski
year                               2021
population                        20717
powiat_numer                       2005
name_normalized_pop           hajnowski
wiek                             Ogółem
powiat_nazwa                  hajnowski
wiek_eurostat                     TOTAL
deaths                              428
name_normalized_tot           hajnowski
CMR                         2065.936188
Name: 7147, dtype: object

In [68]:
population_df.to_csv('../results/population_TOTAL_FEMALES.csv', index=False)